In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!pip install keras_metrics
import keras_metrics as km

In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [5]:
os.listdir(root_path+'/non_english_data')

['Greek.zip',
 'Danish.zip',
 'Turkish.zip',
 'Greek',
 'Danish',
 'Turkish',
 'Arabic.zip',
 'readme-data-ar.txt',
 'offenseval-ar-dev-v1.tsv',
 'offenseval-ar-training-v1.tsv']

In [0]:
train_data=pd.read_csv(root_path+'/non_english_data/offenseval-ar-training-v1.tsv',delimiter='\t')

In [0]:
dev_data=pd.read_csv(root_path+'/non_english_data/offenseval-ar-dev-v1.tsv',delimiter='\t')

In [8]:
print(train_data['tweet'][2])

RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ان تهلك بني سعود المجرمين. لاجل اطفال اليمن شاركوا.


In [9]:
print(train_data.shape)

(6839, 3)


## Cleaning


In [10]:
!pip install cltk

     |████████████████████████████████| 614kB 9.1MB/s 
     |████████████████████████████████| 460kB 23.0MB/s 
     |████████████████████████████████| 757kB 31.4MB/s 
     |████████████████████████████████| 1.5MB 50.7MB/s 
     |████████████████████████████████| 471kB 53.9MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
  Created wheel for cltk: filename=cltk-0.1.113-cp36-none-any.whl size=696129 sha256=a9fe88a5077a4a7c04fc45123a3dc002dc624a0a3beb4aaefe7cffa0b3b09986
  Stored in directory: /root/.cache/pip/wheels/cb/da/9b/b4a9924af2c345d40e8ea7aea34a05a8f31dbd2f8357126ce3
Successfully built cltk


In [0]:
from cltk.phonology.arabic.romanization import transliterate
mode = 'buckwalter'
ar_string = 'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [0]:
ignore = ''

In [0]:
reverse = True

In [14]:
transliterate(mode, ar_string, ignore, reverse)

'bisomi Allhi Alra~Hom`ni Alra~Hiyomi'

In [0]:
def translit(s):
  t=transliterate(mode,s,ignore,reverse)
  return t

In [0]:
train_data['trans']=train_data['tweet'].apply(translit)

In [17]:
train_data['trans']

0       AlHmdllh yArb fwz mhm yA zmAlk.. kl AldEm lykm...
1                 fdwh yA bxt fdwh yA zmn wAHd mnkm yjybh
2       RT @USER: yA rb yA wAHd yA >Hd bHq ywm AlAHd A...
3       RT @USER: #hwA_AlHryp yA wjE qlby Elyky yA Amy...
4               yA bkwn bHyAtk Al>hm yA <mA mA bdy >kwn 🎼
                              ...                        
6834    @USER yA HmAr ، yA jAhl ، nsbp AlbATl mA bttHs...
6835    RT @USER: @USER kl zq yA TAqyp yA wATy yA Hqyr...
6836    @USER<LF>yA kbyr yA mmtE yA njm lAbd >n tEy jy...
6837          yA rb AlAtHAd yfwz yA rb. 😭😭 #AlAtHAd_AlnSr
6838    yEz ElyA Adxl Al$ArE wmlAqyk$ wAqf mstnyny fy ...
Name: trans, Length: 6839, dtype: object

## Tokenization and Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [0]:
df_train,df_dev=train_test_split(train_data,test_size=0.1)

In [0]:
y_train=(df_train['subtask_a']=="OFF").astype(int)
y_dev=(df_dev['subtask_a']=="OFF").astype(int)

In [22]:
print(train_data.shape)

(6839, 4)


In [0]:
max_words =2000
max_len = 50
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['tweet'].astype(str))

In [120]:
train_data['tweet']

0       الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم...
1                 فدوه يا بخت فدوه يا زمن واحد منكم يجيبه
2       RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ا...
3       RT @USER: #هوا_الحرية يا وجع قلبي عليكي يا امي...
4               يا بكون بحياتك الأهم يا إما ما بدي أكون 🎼
                              ...                        
6834    @USER يا حمار ، يا جاهل ، نسبة الباطل ما بتتحس...
6835    RT @USER: @USER كل زق يا طاقية يا واطي يا حقير...
6836    @USER<LF>يا كبير يا ممتع يا نجم لابد أن تعي جي...
6837          يا رب الاتحاد يفوز يا رب. 😭😭 #الاتحاد_النصر
6838    يعز عليا ادخل الشارع وملاقيكش واقف مستنيني في ...
Name: tweet, Length: 6839, dtype: object

In [0]:
from cltk.corpus.arabic.utils.pyarabic import araby

In [0]:
# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
# sequences_dev = tok.texts_to_sequences(df_dev['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Model

In [0]:
word_index=tok.word_index

In [126]:
print(word_index)

{'يا': 1, 'lf': 2, 'user': 3, 'و': 4, 'rt': 5, 'من': 6, 'الله': 7, 'url': 8, 'في': 9, 'ما': 10, 'رب': 11, 'لا': 12, 'كل': 13, '،': 14, 'على': 15, 'انت': 16, 'اللهم': 17, 'انا': 18, 'اللي': 19, 'ولا': 20, 'بس': 21, 'والله': 22, 'قلبي': 23, 'عليك': 24, 'مش': 25, 'ان': 26, 'يارب': 27, 'عن': 28, 'علي': 29, 'حبيبي': 30, 'هو': 31, 'لو': 32, 'لك': 33, 'مع': 34, 'ايه': 35, 'ابن': 36, 'وانت': 37, 'هذا': 38, 'كان': 39, 'يوم': 40, '؟': 41, 'لي': 42, 'محمد': 43, 'ربنا': 44, 'بعد': 45, 'ده': 46, 'أنا': 47, 'صباح': 48, 'الناس': 49, 'ابو': 50, 'دي': 51, 'طيب': 52, 'كريم': 53, 'ف': 54, 'غير': 55, '😂': 56, 'رحيم': 57, 'الي': 58, 'not': 59, 'الاتحاد': 60, 'الخير': 61, 'له': 62, 'يعني': 63, 'ع': 64, 'فى': 65, 'عشان': 66, 'فيك': 67, 'لما': 68, 'أنت': 69, '❤️': 70, 'النصر': 71, 'عمري': 72, 'منك': 73, 'او': 74, 'فيه': 75, 'قبل': 76, 'ذا': 77, 'انتي': 78, 'بن': 79, 'الدنيا': 80, 'أن': 81, 'حتى': 82, 'كنت': 83, 'شي': 84, 'مصر': 85, 'الهلال': 86, 'الا': 87, 'لنا': 88, 'ويا': 89, 'عليه': 90, 'روحي': 91, 'خير': 

In [127]:
!pip install keras-self-attention

In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    x1=Embedding(max_words+1,embed_size)(inp1)
    #x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
    #              trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    #x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = CuDNNLSTM(100, return_sequences=True)(x1)   
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=100)

In [0]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

In [132]:
model_bi.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 50, 100)      200100      input_6[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)        (None, 50, 100)      80800       embedding_6[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_6 (GlobalM (None, 100)          0           cu_dnnlstm_6[0][0]               
____________________________________________________________________________________________

In [0]:
y=(train_data['subtask_a']=="OFF").astype(int)

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y),y)
class_weights=dict(enumerate(class_weights))

In [0]:
# print(class_weights)

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_arabic.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
# model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_dev),epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])

In [0]:
# y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

# print(classification_report(y_dev, y_preds_dl.round()))

In [0]:
# train_data.head()

In [0]:

# sequences_train = tok.texts_to_sequences(df_train['tweet'].astype(str))
# vocab_size = len(tok.word_index) + 1
# sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [142]:
f1s=[]
for train_ind,dev_ind in kfold.split(train_data['tweet'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['tweet'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['tweet'][dev_ind]
  y_dev=y[dev_ind]
  seq_train=tok.texts_to_sequences(x_train.astype(str))
  seq_matrix_train=sequence.pad_sequences(seq_train,maxlen=max_len,padding='post',truncating='post')
  seq_dev=tok.texts_to_sequences(x_dev.astype(str))
  seq_matrix_dev=sequence.pad_sequences(seq_dev,maxlen=max_len,padding='post',truncating='post')
  model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])
  model_bi.fit([seq_matrix_train],y_train,validation_split=0.1,epochs=10,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])
  scores = model_bi.evaluate(seq_matrix_dev, y_dev, verbose=0)
  pred = model_bi.predict(seq_matrix_dev)
  print(classification_report(y_dev, pred.round()))
  f1s.append(f1_score(y_dev,pred.round(),average='macro'))
  #f1s.append(scores[2])
  print(scores)
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s),np.std(f1s)))

Train on 5539 samples, validate on 616 samples
Epoch 1/10
5539/5539 [==============================] - 4s 680us/step - loss: 0.6300 - acc: 0.5635 - f1_score: 0.3885 - val_loss: 0.4576 - val_acc: 0.8620 - val_f1_score: 0.6559
Epoch 2/10
5539/5539 [==============================] - 2s 341us/step - loss: 0.3949 - acc: 0.8391 - f1_score: 0.6689 - val_loss: 0.3928 - val_acc: 0.8766 - val_f1_score: 0.7031
Epoch 3/10
5539/5539 [==============================] - 2s 325us/step - loss: 0.2823 - acc: 0.8850 - f1_score: 0.7509 - val_loss: 0.3973 - val_acc: 0.8214 - val_f1_score: 0.6667
Epoch 4/10
5539/5539 [==============================] - 2s 327us/step - loss: 0.2212 - acc: 0.9050 - f1_score: 0.7945 - val_loss: 0.5158 - val_acc: 0.8799 - val_f1_score: 0.7176
Epoch 5/10
5539/5539 [==============================] - 2s 346us/step - loss: 0.1793 - acc: 0.9269 - f1_score: 0.8358 - val_loss: 0.6650 - val_acc: 0.8133 - val_f1_score: 0.6395
Epoch 6/10
5539/5539 [==============================] - 2s 322u

In [0]:

from sklearn.metrics import f1_score

In [47]:
 pred = model_bi.predict(seq_matrix_dev)
print(f1_score(y_dev,pred.round(),average='macro'))
print(classification_report(y_dev, pred.round()))

0.9412888806601777
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       546
           1       0.89      0.92      0.91       137

    accuracy                           0.96       683
   macro avg       0.94      0.95      0.94       683
weighted avg       0.96      0.96      0.96       683



## ML algos

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?' ]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [49]:
vectorizer =TfidfVectorizer(max_features=1000,lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,5),dtype=np.float32,max_df=0.5,min_df=2)
vectorizer.fit(train_data['trans'].astype(str))
# x_train=vectorizer.transform(['clean_col'].astype(str))
# x_dev=vectorizer.transform(dev_data['clean_col'].astype(str))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=1000,
                min_df=2, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(x_train, labels_train_le)
# y_preds=clf.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [0]:
from sklearn.linear_model import SGDClassifier
# sgd_clf=SGDClassifier().fit(x_train,labels_train_le)
# y_preds=sgd_clf.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [0]:
from sklearn.svm import SVC
# clf_svc=SVC(gamma='scale',decision_function_shape='ovo').fit(x_train,labels_train_le)
# y_preds=clf_svc.predict(x_dev)
# f1_score(labels_dev_le,y_preds,average='macro')

In [54]:
f1s_nb=[]
f1s_sgd=[]
f1s_svc=[]
for train_ind,dev_ind in kfold.split(train_data['trans'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['trans'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['trans'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  clf = MultinomialNB().fit(x_train, y_train)
  y_preds=clf.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_nb.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  print(f1_score(y_dev,y_preds,average='macro'))
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_nb),np.std(f1s_nb)))

0.6979736376155814
0.6919929175683652
0.6971512825031918
0.7256822017612754
0.6936721754860464
0.6450901543650279
0.7201016925541548
0.717697887212631
0.71591579775723
0.7176295696843883
Mean f1:0.7022907316507891, standard dev:0.022405302472581257


In [55]:
for train_ind,dev_ind in kfold.split(train_data['trans'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['trans'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['trans'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  sgd_clf=SGDClassifier().fit(x_train,y_train)
  y_preds=sgd_clf.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_sgd.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  #print(scores)
  print(f1_score(y_dev,y_preds,average='macro'))
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_sgd),np.std(f1s_sgd)))

0.7221816976127321
0.7398375285766334
0.7358697640387781
0.7562642737977615
0.733827065420867
0.7074759437453737
0.7389258226202209
0.7581051073279053
0.7243523316062177
0.7654916451391387
Mean f1:0.7382331179885628, standard dev:0.01699242964028436


In [59]:
f1s_svc=[]
for train_ind,dev_ind in kfold.split(train_data['trans'],y):
  #print(train_data['tweet'][train_ind].shape)
  x_train=train_data['trans'][train_ind]
  y_train=y[train_ind]
  x_dev=train_data['trans'][dev_ind]
  y_dev=y[dev_ind]
  x_train=vectorizer.transform(x_train.astype(str))
  x_dev=vectorizer.transform(x_dev.astype(str))
  clf_svc=SVC(gamma='scale',decision_function_shape='ovo').fit(x_train,y_train)
  y_preds=clf_svc.predict(x_dev)
  #print(f1_score(y_dev,y_preds,average='macro'))
  f1s_svc.append(f1_score(y_dev,y_preds,average='macro'))
  #f1s.append(scores[2])
  print(f1_score(y_dev,y_preds,average='macro'))
print("Mean f1:{0}, standard dev:{1}".format(np.mean(f1s_svc),np.std(f1s_svc)))

0.6860121975211488
0.6849489795918368
0.692696401802435
0.7342657342657343
0.6961105052708106
0.6383344348975545
0.7242497409910398
0.717697887212631
0.6951039790404454
0.6922404914294122
Mean f1:0.6961660352023049, standard dev:0.025111547736619735
